## Análisis Exploratorio de Datos (EDA)

In [17]:
# Importar librerías necesarias
import pandas as pd
import sqlite3 as sql

### Cargar datos

In [18]:
import pandas as pd
import sqlite3 as sql

ruta_archivo = "../data/raw/adidas_us_sales_dataset.xlsx"
df = pd.read_excel(ruta_archivo, index_col=None)

# Eliminar columna de índice anterior si existe
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)

# Guardar en SQLite
conn = sql.connect("../data/raw/adidas_sales.db")
df.to_sql("ventas_adidas", conn, if_exists="replace", index=False)

print("Base de datos creada con éxito")

cursor = conn.cursor()


Base de datos creada con éxito


## Tamaño del Dataset

In [19]:
df

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,1185732,1969-12-31 19:00:43.831,Northeast,New York,New York,Men's Street Footwear,50.0,1200,600000.0,300000.00,0.50,In-store
1,Foot Locker,1185732,1969-12-31 19:00:43.832,Northeast,New York,New York,Men's Athletic Footwear,50.0,1000,500000.0,150000.00,0.30,In-store
2,Foot Locker,1185732,1969-12-31 19:00:43.833,Northeast,New York,New York,Women's Street Footwear,40.0,1000,400000.0,140000.00,0.35,In-store
3,Foot Locker,1185732,1969-12-31 19:00:43.834,Northeast,New York,New York,Women's Athletic Footwear,45.0,850,382500.0,133875.00,0.35,In-store
4,Foot Locker,1185732,1969-12-31 19:00:43.835,Northeast,New York,New York,Men's Apparel,60.0,900,540000.0,162000.00,0.30,In-store
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9643,Foot Locker,1185732,1969-12-31 19:00:44.220,Northeast,New Hampshire,Manchester,Men's Apparel,50.0,64,3200.0,896.00,0.28,Outlet
9644,Foot Locker,1185732,1969-12-31 19:00:44.220,Northeast,New Hampshire,Manchester,Women's Apparel,41.0,105,4305.0,1377.60,0.32,Outlet
9645,Foot Locker,1185732,1969-12-31 19:00:44.249,Northeast,New Hampshire,Manchester,Men's Street Footwear,41.0,184,7544.0,2791.28,0.37,Outlet
9646,Foot Locker,1185732,1969-12-31 19:00:44.249,Northeast,New Hampshire,Manchester,Men's Athletic Footwear,42.0,70,2940.0,1234.80,0.42,Outlet


Tamaño del dataset: 9648 Filas y 13 columnas

## Tipos de variables detectadas

In [29]:
df.dtypes

Retailer                    object
Retailer ID                  int64
Invoice Date        datetime64[ns]
Region                      object
State                       object
City                        object
Product                     object
Price per Unit             float64
Units Sold                   int64
Total Sales                float64
Operating Profit           float64
Operating Margin           float64
Sales Method                object
dtype: object

#### Numéricas:

- Retailer ID
- Price per Unit
- Units Sold
- Total Sales
- Operating Profit
- Operating Margin

#### Categóricas:

- Retailer
- Region
- State
- City
- Product
- Sales Method

#### Fechas:

- Invoice Date

## Exploración Inicial y calidad de datos:

### Columnas con valores nulos:

In [22]:
valores_nulos = """
                SELECT
                    SUM(CASE WHEN Retailer IS NULL THEN 1 ELSE 0 END) AS retailer_nulos,
                    SUM(CASE WHEN `Retailer ID` IS NULL THEN 1 ELSE 0 END) AS retailerid_nulos,
                    SUM(CASE WHEN `Invoice Date` IS NULL THEN 1 ELSE 0 END) AS invoice_date_nulos,
                    SUM(CASE WHEN Region IS NULL THEN 1 ELSE 0 END) AS region_nulos,
                    SUM(CASE WHEN State IS NULL THEN 1 ELSE 0 END) AS state_nulos,
                    SUM(CASE WHEN City IS NULL THEN 1 ELSE 0 END) AS city_nulos,
                    SUM(CASE WHEN Product IS NULL THEN 1 ELSE 0 END) AS products_nulos,
                    SUM(CASE WHEN `Price per Unit` IS NULL THEN 1 ELSE 0 END) AS price_per_unit_nulos,
                    SUM(CASE WHEN `Units Sold` IS NULL THEN 1 ELSE 0 END) AS units_sold_nulos,
                    SUM(CASE WHEN `Total Sales` IS NULL THEN 1 ELSE 0 END) AS total_sales_nulos,
                    SUM(CASE WHEN `Operating Profit` IS NULL THEN 1 ELSE 0 END) AS operating_profit_nulos,
                    SUM(CASE WHEN `Operating Margin` IS NULL THEN 1 ELSE 0 END) AS operating_margin_nulos,
                    SUM(CASE WHEN `Sales Method` IS NULL THEN 1 ELSE 0 END) AS sales_methond_nulos
                FROM ventas_adidas;
                """
valores_nulos = pd.read_sql_query(valores_nulos, conn)
valores_nulos

,retailer_nulos,retailerid_nulos,invoice_date_nulos,region_nulos,state_nulos,city_nulos,products_nulos,price_per_unit_nulos,units_sold_nulos,total_sales_nulos,operating_profit_nulos,operating_margin_nulos,sales_methond_nulos
0,0,0,0,0,0,0,0,0,0,0,0,0,0


En conclusión no hay valores nulos

### Registros duplicados

In [51]:
consulta = """
            WITH duplicados AS (
                SELECT *,
                    ROW_NUMBER() OVER (
                        PARTITION BY `Retailer ID`, `Invoice Date`, Product, `Sales Method`, City, State, `Price per Unit`
                        ORDER BY `Units Sold` DESC
                    ) AS num_fila
                FROM ventas_adidas
            )
            SELECT *
            FROM duplicados
            WHERE num_fila = 2;
            """
duplicados = pd.read_sql_query(consulta, conn)
duplicados

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method,num_fila
0,West Gear,1185732,1969-12-31 19:00:43.945000,Southeast,Florida,Miami,Men's Athletic Footwear,55.0,775,426250.0,149187.5,0.35,In-store,2
1,West Gear,1185732,1969-12-31 19:00:43.945000,Southeast,Florida,Miami,Men's Athletic Footwear,54.0,194,10476.0,5238.0,0.50,Online,2
2,Foot Locker,1185732,1969-12-31 19:00:44.245000,South,Mississippi,Jackson,Women's Athletic Footwear,25.0,91,2275.0,1137.5,0.50,Online,2
3,Sports Direct,1185732,1969-12-31 19:00:44.263000,Midwest,North Dakota,Fargo,Women's Athletic Footwear,25.0,22,550.0,231.0,0.42,Online,2
4,Amazon,1185732,1969-12-31 19:00:44.264000,Midwest,Ohio,Columbus,Women's Street Footwear,20.0,124,2480.0,1314.4,0.53,Online,2
5,Sports Direct,1185732,1969-12-31 19:00:44.289000,Southeast,North Carolina,Charlotte,Women's Street Footwear,20.0,144,2880.0,1584.0,0.55,Online,2
6,Kohl's,1197831,1969-12-31 19:00:44.241000,Midwest,Kansas,Wichita,Women's Athletic Footwear,20.0,42,840.0,378.0,0.45,Online,2
7,Kohl's,1197831,1969-12-31 19:00:44.267000,Midwest,Kansas,Wichita,Men's Street Footwear,20.0,153,3060.0,1499.4,0.49,Online,2
8,Kohl's,1197831,1969-12-31 19:00:44.267000,Midwest,Kansas,Wichita,Women's Athletic Footwear,20.0,26,520.0,249.6,0.48,Online,2
9,Walmart,1197831,1969-12-31 19:00:44.274000,South,Arkansas,Little Rock,Men's Athletic Footwear,20.0,98,1960.0,882.0,0.45,Online,2


In [52]:
consulta = """
            SELECT DISTINCT `Retailer ID`, `Invoice Date`, Product, `Sales Method`, City, State, `Price per Unit`
            FROM ventas_adidas;
           """
valores_distintos = pd.read_sql_query(consulta, conn)
valores_distintos

,Retailer ID,Invoice Date,Product,Sales Method,City,State,Price per Unit
0,1185732,1969-12-31 19:00:43.831000,Men's Street Footwear,In-store,New York,New York,50.0
1,1185732,1969-12-31 19:00:43.832000,Men's Athletic Footwear,In-store,New York,New York,50.0
2,1185732,1969-12-31 19:00:43.833000,Women's Street Footwear,In-store,New York,New York,40.0
3,1185732,1969-12-31 19:00:43.834000,Women's Athletic Footwear,In-store,New York,New York,45.0
4,1185732,1969-12-31 19:00:43.835000,Men's Apparel,In-store,New York,New York,60.0
...,...,...,...,...,...,...,...
9629,1185732,1969-12-31 19:00:44.220000,Men's Apparel,Outlet,Manchester,New Hampshire,50.0
9630,1185732,1969-12-31 19:00:44.220000,Women's Apparel,Outlet,Manchester,New Hampshire,41.0
9631,1185732,1969-12-31 19:00:44.249000,Men's Street Footwear,Outlet,Manchester,New Hampshire,41.0
9632,1185732,1969-12-31 19:00:44.249000,Men's Athletic Footwear,Outlet,Manchester,New Hampshire,42.0
